In [6]:


import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from scipy.stats import spearmanr, pearsonr
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import matplotlib.dates as mdates

# If running for the first time, uncomment the following line:
# nltk.download('vader_lexicon')

# ------------------------
# Debug Printing Function
# ------------------------
def debug_print(df, message, head=True):
    print("\n" + "="*60)
    print(message)
    print("-"*60)
    print("Shape:", df.shape)
    if head:
        print(df.head())
    print(df.info())
    print("="*60 + "\n")

# ------------------------
# 1. Load and Preprocess Tweets
# ------------------------
print("Loading Tweets CSV...")
tweets_data = pd.read_csv('Tesla.csv')
print("Tweets loaded. Shape:", tweets_data.shape)

# Convert 'date' to datetime with UTC timezone
tweets_data['timestamp'] = pd.to_datetime(tweets_data['date'], utc=True, errors='coerce')
tweets_data['hour'] = tweets_data['timestamp'].dt.floor('H')
debug_print(tweets_data, "Tweets Data After Timestamp Conversion")

# Create a new DataFrame to hold cleaned data and sentiment
cleaned_data = pd.DataFrame()
cleaned_data['original_tweet'] = tweets_data['tweet']
cleaned_data['timestamp'] = tweets_data['timestamp']
cleaned_data['hour'] = tweets_data['hour']
cleaned_data['cleaned_tweet'] = ""
cleaned_data['compound_score'] = 0.0

# Preprocess tweets function
def preprocess_tweet(tweet):
    tweet = str(tweet)
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'@\w+', '', tweet)                     # Remove mentions
    tweet = re.sub(r'#', '', tweet)                        # Remove hashtags
    tweet = re.sub(r'[^\w\s]', '', tweet)                  # Remove special characters
    tweet = re.sub(r'\d+', '', tweet)                      # Remove numbers
    tweet = tweet.strip()                                  # Remove leading/trailing whitespace
    return tweet.lower()

cleaned_data['cleaned_tweet'] = cleaned_data['original_tweet'].apply(preprocess_tweet)

# Sentiment analysis
sentiment_analyzer = SentimentIntensityAnalyzer()
cleaned_data['compound_score'] = cleaned_data['cleaned_tweet'].apply(lambda x: sentiment_analyzer.polarity_scores(x)['compound'])

debug_print(cleaned_data, "Cleaned Data with Compound Scores")

# Aggregate hourly sentiment scores
hourly_sentiment = cleaned_data.groupby('hour')['compound_score'].mean().reset_index()
hourly_sentiment.rename(columns={'compound_score': 'average_compound'}, inplace=True)
debug_print(hourly_sentiment, "Hourly Sentiment Aggregation")

# ------------------------
# 2. Load and Process Stock Data
# ------------------------
print("Loading Stock Data CSV...")
stock_data = pd.read_csv('TSLA_stock_data.csv')
print("Stock data loaded. Shape:", stock_data.shape)

stock_data.rename(columns={'Time': 'time', 'Price': 'Close'}, inplace=True)
debug_print(stock_data, "Stock Data After Renaming Columns")

# Extract the common date from the tweets dataset (assuming all tweets are from the same date)
if not tweets_data.empty:
    common_date = pd.to_datetime(tweets_data['date'].iloc[0], errors='coerce').date()
    print("Common date extracted from tweets:", common_date)
else:
    print("Tweets data is empty, cannot extract common date.")
    common_date = pd.to_datetime('2022-07-11').date()  # fallback if empty

# The times are in 24-hour format in TSLA_stock_data.csv now, so we can parse directly
stock_data['timestamp'] = pd.to_datetime(
    stock_data['time'].apply(lambda t: f"{common_date} {t}"),
    format='%Y-%m-%d %H:%M',
    utc=True,
    errors='coerce'
)
stock_data['hour'] = stock_data['timestamp'].dt.floor('H')
stock_data['Close'] = pd.to_numeric(stock_data['Close'], errors='coerce')
stock_data['Hourly_Return'] = stock_data['Close'].pct_change()

debug_print(stock_data, "Stock Data After Timestamp & Hourly_Return Calculation")

# ------------------------
# 3. Align Data
# ------------------------
merged_data = pd.merge(hourly_sentiment, stock_data, on='hour', how='inner')

debug_print(merged_data, "Merged Data")

# ------------------------
# 4. Compute Spearman and Pearson Correlation
# ------------------------
# Check variation in average_compound and Hourly_Return
print("Unique average_compound values:", merged_data['average_compound'].unique())
print("Unique Hourly_Return values:", merged_data['Hourly_Return'].unique())

if not merged_data.empty and merged_data['average_compound'].nunique() > 1 and merged_data['Hourly_Return'].nunique() > 1:
    spearman_corr, spearman_p = spearmanr(merged_data['average_compound'], merged_data['Hourly_Return'])
    pearson_corr, pearson_p = pearsonr(merged_data['average_compound'], merged_data['Hourly_Return'])
    print(f"Spearman Correlation: {spearman_corr}, p-value: {spearman_p}")
    print(f"Pearson Correlation: {pearson_corr}, p-value: {pearson_p}")
else:
    print("Insufficient or non-overlapping data for meaningful correlation.")
    # Print reasons why
    if merged_data.empty:
        print("Reason: merged_data is empty. No overlapping hours.")
    else:
        if merged_data['average_compound'].nunique() <= 1:
            print("Reason: average_compound has no variation.")
        if merged_data['Hourly_Return'].nunique() <= 1:
            print("Reason: Hourly_Return has no variation.")

# ------------------------
# 5. Visualizations
# ------------------------
sns.set_style("whitegrid")

if not hourly_sentiment.empty:
    # A) Line plot of average hourly sentiment over time
    plt.figure(figsize=(12,6))
    plt.plot(hourly_sentiment['hour'], hourly_sentiment['average_compound'], marker='o', color='blue')
    plt.title('Average Hourly Sentiment (Compound Score)')
    plt.xlabel('Hour')
    plt.ylabel('Average Compound Sentiment')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if not stock_data.empty:
    # B) Line plot of TSLA stock price over time
    plt.figure(figsize=(12,6))
    plt.plot(stock_data['timestamp'], stock_data['Close'], marker='o', color='green')
    plt.title('TSLA Stock Price Over Time')
    plt.xlabel('Time')
    plt.ylabel('Price (Close)')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if not hourly_sentiment.empty and not stock_data.empty:
    # C) Dual-axis line plot: Average Sentiment and Stock Close Price Over Time
    fig, ax1 = plt.subplots(figsize=(12,6))

    color = 'tab:blue'
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Avg Sentiment', color=color)
    ax1.plot(hourly_sentiment['hour'], hourly_sentiment['average_compound'], marker='o', color=color, label='Avg Sentiment')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:red'
    # Need to align stock_data with hourly_sentiment time range
    aligned_stock = stock_data[stock_data['hour'].isin(hourly_sentiment['hour'])]
    ax2.set_ylabel('TSLA Close', color=color)
    ax2.plot(aligned_stock['timestamp'], aligned_stock['Close'], marker='o', color=color, label='TSLA Close Price')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()
    plt.title('Sentiment vs. Stock Price Over Time')
    plt.show()

if not cleaned_data.empty:
    # D) Histogram of Compound Scores
    plt.figure(figsize=(12,6))
    sns.histplot(cleaned_data['compound_score'], bins=20, kde=True, color='purple')
    plt.title('Distribution of Compound Sentiment Scores')
    plt.xlabel('Compound Score')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

if not merged_data.empty:
    # E) Scatter plot comparing Average Compound vs Hourly Return
    plt.figure(figsize=(10,6))
    sns.scatterplot(data=merged_data, x='average_compound', y='Hourly_Return', hue='Hourly_Return', palette='coolwarm', size='Hourly_Return', sizes=(50, 200))
    plt.title('Average Sentiment vs. Hourly Return')
    plt.xlabel('Average Compound Sentiment')
    plt.ylabel('Hourly Return (%)')
    plt.tight_layout()
    plt.show()

# F) Create a word cloud of the cleaned tweets
if not cleaned_data.empty:
    all_words = " ".join(cleaned_data['cleaned_tweet'])
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords, max_words=200).generate(all_words)

    plt.figure(figsize=(15,7.5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud of Tweets', fontsize=20)
    plt.tight_layout()
    plt.show()

if not cleaned_data.empty:
    # G) Bar plot of most common words
    word_list = all_words.split()
    common_words = Counter(word_list).most_common(20)
    common_words_df = pd.DataFrame(common_words, columns=['word', 'count'])

    plt.figure(figsize=(12,8))
    sns.barplot(x='count', y='word', data=common_words_df, palette='Blues_r')
    plt.title('Top 20 Most Common Words in Tweets')
    plt.xlabel('Count')
    plt.ylabel('Word')
    plt.tight_layout()
    plt.show()

if not stock_data.empty:
    # H) Line plot showing Hourly Return vs. Time
    plt.figure(figsize=(12,6))
    plt.plot(stock_data['timestamp'], stock_data['Hourly_Return']*100, marker='o', color='red')
    plt.title('TSLA Hourly Return Over Time')
    plt.xlabel('Time')
    plt.ylabel('Hourly Return (%)')
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if not merged_data.empty:
    # I) Correlation heatmap
    plt.figure(figsize=(5,4))
    corr = merged_data[['average_compound','Hourly_Return']].corr(method='spearman')
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Spearman Correlation Heatmap')
    plt.tight_layout()
    plt.show()


Loading Tweets CSV...
Tweets loaded. Shape: (10016, 39)

Tweets Data After Timestamp Conversion
------------------------------------------------------------
Shape: (10016, 40)
   Unnamed: 0                   id      conversation_id    created_at  \
0           0  1546541426317590528  1545826164564000768  1.657559e+12   
1           1  1546541415857102850  1545826164564000768  1.657559e+12   
2           2  1546541411897581568  1546541411897581568  1.657559e+12   
3           3  1546541379110805508  1546340000500813824  1.657559e+12   
4           4  1546541363470028800  1546541363470028800  1.657559e+12   

                  date  timezone place  \
0  2022-07-11 17:06:24         0   NaN   
1  2022-07-11 17:06:21         0   NaN   
2  2022-07-11 17:06:20         0   NaN   
3  2022-07-11 17:06:12         0   NaN   
4  2022-07-11 17:06:09         0   NaN   

                                               tweet language  \
0  @GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...       en   
1 

ValueError: array must not contain infs or NaNs